Загрузка файла

In [1]:
import pandas as pd



In [4]:
df = pd.read_csv('https://drive.google.com/file/d/1-GxCKCv8VhXvzH4tH-KstqPsLCNt8hQH/view?usp=share_link', sep=',')
idc=pd.DataFrame(df.reco_id_curr)
df=df.drop(columns='reco_id_curr')


AttributeError: ignored

Устраняем проблемы с категорийными столбами

In [ ]:
one_hot = pd.get_dummies(df['gender'])
one_hot.rename(columns={"M":"gender"},inplace=True)
one_hot = one_hot.drop(columns=['F'])

df = df.drop('gender',axis = 1)
df = df.join(one_hot)

df = df.drop('own_car_flag',axis = 1)
df.age_own_car=df.age_own_car.fillna(-100)

one_hot = pd.get_dummies(df['own_realty_flag'])
one_hot.rename(columns={"Y":"own_realty_flag"},inplace=True)
one_hot = one_hot.drop(columns=['N'])

df = df.drop('own_realty_flag',axis = 1)
df = df.join(one_hot)

one_hot = pd.get_dummies(df['contract_type_name'])
one_hot = one_hot.drop(columns=['Revolving loans'])

df = df.drop('contract_type_name',axis = 1)
df = df.join(one_hot)

one_hot = pd.get_dummies(df['emergency_state_mode'])
one_hot.rename(columns={"Yes":"emergency_state_modeg"},inplace=True)
one_hot = one_hot.drop(columns=['No'])

df = df.drop('emergency_state_mode',axis = 1)
df = df.join(one_hot)


In [ ]:
for col in df.columns:
  if df[col].dtype==object:
    one_hot = pd.get_dummies(df[col])
    df = df.drop(col,axis = 1)
    df = df.join(one_hot)



```
# Выбран кодовый формат
```
Заполняем

In [ ]:
df.last_phone_number_change=df.last_phone_number_change.fillna(0) #одна лишь пропущена
df.requests_bki_hour=df.requests_bki_hour.fillna(0) 
df.requests_bki_day=df.requests_bki_day.fillna(0)
df.requests_bki_week=df.requests_bki_week.fillna(0)
df.requests_bki_month=df.requests_bki_month.fillna(0)
df.requests_bki_qrt=df.requests_bki_qrt.fillna(0)
df.requests_bki_year=df.requests_bki_year.fillna(0)

df.social_circle_defaults_60_days=df.social_circle_defaults_60_days.fillna(0)
df.observes_60_count_social_circle=df.observes_60_count_social_circle.fillna(0)
df.social_circle_defaults_30_days=df.social_circle_defaults_30_days.fillna(0)
df.observes_30_count_social_circle=df.observes_30_count_social_circle.fillna(0)

In [ ]:
for i in range (27,73):
  m=df[df.columns[i]].median()
  df[df.columns[i]]=df[df.columns[i]].fillna(m)


In [ ]:
m=df.annuity_payment.median()
df.annuity_payment=df.annuity_payment.fillna(m)

m=df.goods_price.median()
df.goods_price=df.goods_price.fillna(m)

m=df.family_members__count.median()
df.family_members__count=df.family_members__count.fillna(m)
 

Шкалирование

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:

X=df.copy()

scaler = preprocessing.StandardScaler()
scaler.fit(X) 
X_scale = scaler.transform(X)
X_scale = pd.DataFrame(X_scale) 
X3=X_scale.copy()


In [ ]:
X3.columns+=1
X3.columns

RangeIndex(start=1, stop=237, step=1)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2



In [ ]:
dropcolumns= [125,26,29,105,25,119,223,30,123,149,8,27,145,140,235,228,208,83,128,7,14,132,150,80,120,20,19,226,129,139,171,28,141,16,144,13,36,64,151,173,134,155,50,220,130,157,10]

pcacolumns=X3.drop(columns=dropcolumns)
constcolunms=X3.drop(columns=pcacolumns.columns)
pcacolumns2=constcolunms.drop(columns=[28,29,30,125,149,105,128,123])
constcolunms=constcolunms.drop(columns=pcacolumns2.columns)


PCA

In [ ]:
n_comp = 3
pca = PCA(n_components = n_comp) 
pca2=pd.DataFrame(pca.fit_transform(pcacolumns2),columns=[356,357,358])

In [ ]:
fortrain=constcolunms.join(pca2)

In [ ]:
pip install catboost 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.2 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostRegressor
from catboost import CatBoost

Загрузка модели и предсказание

In [ ]:
model = CatBoostRegressor()
model.load_model("https://drive.google.com/drive/folders/1laQ0svE8aAKlj25FBNgvLGReBBQXSvZF?usp=share_link/model5.cbm")


In [ ]:
ts_preds = model.predict(fortrain)


In [ ]:
t=pd.DataFrame(ts_preds, columns=['target'])
final=idc.join(t)
final
compression_opts = dict(method='zip',
                        archive_name='answer.csv')  
final.to_csv('answer.zip', index=False,
          compression=compression_opts)  